In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.layers import *

In [2]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [3]:
"""
    Crude Implementation of how I interpreted the paper on Fractional Max Pooling.
    It works when passing the values directly. In this class I considered that the
    pool_ratio of both the rows and cols will be same.
    And this is for understanding purpose only and hence needs more refinement.
"""

class FractionalMaxPooling(keras.layers.Layer):
    def __init__(self, pool_ratio=None, pseudo_random=True, overlap=False, **kwargs):
        self.pool_ratio = pool_ratio
        self.pseudo_random = pseudo_random
        self.overlap = overlap

        super(FractionalMaxPooling, self).__init__(**kwargs)

    @staticmethod
    def _pseudo_fmp(alpha, n_out, u):
        """
            This function takes inputs as pool_ratio or the alpha value and the 
            output shape of the pooling layer. It generates the sequences for
            pooling based on the alpha values. The sequences are generated 
            based on the form : 
                        a_i = ceiling(alpha(i + u)), alpha=(1, 2), u = (0, 1)
    
        """
        out = np.zeros([n_out])
        for i in range(n_out):
            out[i] = np.ceil(alpha * (i + u))
        out = out.astype(np.int32)
        return out

    @staticmethod
    def _pool_regions(image, alpha, n_out):
        """
            This functions calls the sequence generator and yields the pooling
            regions for the images based on the sequence.
        """
        a_i = FractionalMaxPooling._pseudo_fmp(alpha, n_out, 0)
        b_i = FractionalMaxPooling._pseudo_fmp(alpha, n_out, 0)
        for i in range(1, n_out):
            for j in range(1, n_out):
                im_region = image[(a_i[i-1]) : (a_i[i] - 1), (b_i[j - 1]) : (b_i[j]-1)]
                yield im_region, i, j

    
    @staticmethod
    def _image_segs(image):
        for i in range(image.shape[0]):
            for j in range(image.shape[-1]):
                yield image[i,...,j], i
        # if image.shape[0]:
        #     for j in range(image.shape[-1]):
        #         yield image[:,...,j], 0


    @staticmethod
    def _pool_images(images, pool_ratio):
        alpha = pool_ratio
        n_out = int(images.shape[1] / alpha)

        outss = tf.Variable(tf.zeros([images.shape[0], n_out, n_out, 3]), trainable=True)
        for imgs, m in (FractionalMaxPooling._image_segs(images)):
            for i in range(3):
                output = np.zeros((n_out, n_out))
                for im_region, j, k in FractionalMaxPooling._pool_regions(imgs, alpha, n_out):
                    if im_region.size <= 0:
                        output[j, k] = imgs[j, k]

                    elif im_region.size  > 0:
                        output[j, k] = tf.reduce_max(im_region, axis=(0, 1))

                output = np.reshape(output, (1, n_out, n_out, 1))
                outss[m, 0:n_out, 0:n_out, i].assign(output[0, 0:n_out, 0:n_out, 0])

        return outss

    def call(self, x):
        x = FractionalMaxPooling._pool_images(x, self.pool_ratio)
        return x

    def compute_output_shape(self, input_shape):
        if(input_shape[0]!=None):
            batch_size = input_shape[0]
        else:
            batch_size = input_shape[0]
        
        width = int(input_shape[1] / self.pool_ratio)
        height = int(input_shape[2] / self.pool_ratio)
        channels = input_shape[3]
        return (batch_size, width, height, channels)

    def get_config(self):
        config = {
            "pool_ratio" : self.pool_ratio,
            "pseudo_random" : self.pseudo_random,
            "overlap" : self.overlap
        }
        base_config = super(FractionalMaxPooling, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]

In [4]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

In [5]:
Y = FractionalMaxPooling(1.05)(X_train[0:10])

In [6]:
Y.shape

TensorShape([10, 30, 30, 3])

In [7]:
Z = MaxPooling2D((2,2))(X_train[0:10])

In [8]:
Z.shape

TensorShape([10, 16, 16, 3])